In [107]:
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow as tf

In [108]:
#Get data object
%run 6-transformer_data.ipynb

AttributeError: 'Tokenizer' object has no attribute 'get_vocab_size'

AttributeError: 'Tokenizer' object has no attribute 'get_vocab_size'

In [ ]:
#data_object = TransformerDataSet("tiny_shakespeare", 80)


file_path_for_text = r"C:\Users\willi\Desktop\Data Science Portfolio\shakespearegen\data\raw\archive\alllines.txt"
data_object = TransformerDataSetText(file_path_for_text, 80)

In [ ]:
def positional_encoding(length: int, depth: int):
    effective_depth = depth / 2
    depth_vector = np.repeat(np.arange(effective_depth), 2)
    frequency_vector = 1/(10000**((2* depth_vector)/depth))
    sequence_vector = np.arange(length)
    pos_encoding = sequence_vector.reshape([-1,1]) * frequency_vector.reshape([1,-1])
    pos_encoding[:,::2] = np.sin(pos_encoding[:,::2])
    pos_encoding[:,1::2] = np.cos(pos_encoding[:,1::2])
    return tf.cast(pos_encoding, dtype=tf.float32)

In [109]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, depth):
        super().__init__()
        self.depth = depth
        self.embedding_layer = tf.keras.layers.Embedding(vocab_size, depth, mask_zero=True)
        self.positional_func = positional_encoding(vocab_size, depth)
    
    def call(self, x):
        length = tf.shape(x)[1] #this is for baches
        x = self.embedding_layer(x)
        x *= tf.math.sqrt(tf.cast(self.depth, tf.float32)) #not sure
        x = x + self.positional_func[tf.newaxis, :length, :]
        return x

Inputs and labels for purpose of testing in downstream notebooks

In [110]:
#These are limited to match the batchsize of the actual dataset
input, main_labels = data_object.inputs[:64], data_object.outputs[:64]
#input, main_labels = data_object.inputs, data_object.outputs
vocab_s = data_object.vocab_size

In [111]:
print(vocab_s)

[18880, 1, 397, 5, 6132]


In [91]:
posit_layer_inst = PositionalEmbedding(vocab_size=vocab_s, depth=1024)

TypeError: '<=' not supported between instances of 'list' and 'int'

In [81]:
posit_layer_output = posit_layer_inst(tf.convert_to_tensor(input))
posit_layer_output.shape

TensorShape([10265, 79, 512])

Batched dataset for training

In [71]:
print(len(data_object.laoded_text[0]))

4366288


In [72]:
input_to_tensor = tf.convert_to_tensor(data_object.inputs) 
labels_to_tensor = tf.convert_to_tensor(data_object.outputs)

In [73]:
print(input_to_tensor.shape)
print(labels_to_tensor.shape)

(10265, 79)
(10265, 79)


In [74]:
labels_to_tensor.shape

TensorShape([10265, 79])

In [75]:
input_dataset = tf.data.Dataset.from_tensor_slices(input_to_tensor)
label_dataset = tf.data.Dataset.from_tensor_slices(labels_to_tensor)

In [76]:
tf_main_dataset = tf.data.Dataset.zip((input_dataset, label_dataset))

In [77]:
tf_main_dataset

<_ZipDataset element_spec=(TensorSpec(shape=(79,), dtype=tf.int32, name=None), TensorSpec(shape=(79,), dtype=tf.int32, name=None))>

In [78]:
batched_dataset = tf_main_dataset.batch(128, drop_remainder=True)